## Imports

In [ ]:
import sys 
import numpy as np 
import matplotlib.pyplot as plt


import torch
import torchvision 
import torch.optim as optim
import torch.nn as nn

sys.path.insert(0, '../src_code')

import models
from train import *
from metrics import *  

print("Python: %s" % sys.version)
print("Pytorch: %s" % torch.__version__)

## Hyperparameters

In [ ]:
n_epochs = 30
batch_size = 128
lr = 0.001

## Load CIFAR100

In [ ]:
# define series of transforms to pre process images 
transform = torchvision.transforms.Compose([
    #torchvision.transforms.Pad(2),
    torchvision.transforms.ToTensor(),
    #torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
    

classes = None


# load training set 
trainset = torchvision.datasets.CIFAR100('../datasets/', train=True, transform=transform, download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# load test set 
testset = torchvision.datasets.CIFAR100('../datasets/', train=False, transform=transform, download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# helper function to unnormalize and plot image 
def imshow(img):
    img = np.array(img)
    img = img / 2 + 0.5
    img = np.moveaxis(img, 0, -1)
    plt.imshow(img)
    
# display sample from dataset 
imgs,labels = iter(trainloader).next()
imshow(torchvision.utils.make_grid(imgs))  

## Model Architecture

In [ ]:
# determine device to run network on (runs on gpu if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


multi_layer_perceptron = models.cnn(3, 100, 64)


# load the torchvision resnet18 implementation 
resnet18 = torchvision.models.resnet18(num_classes=100)
# resnet fix for cifar10 image size 
resnet18.avgpool = nn.AvgPool2d(2, stride=1)

#alexnet = torchvision.models.alexnet()


vgg16 = torchvision.models.vgg16(num_classes=100)
# vgg16 fix for cifar10 image size 
vgg16.classifier = nn.Sequential(
            nn.Linear(512, 64),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(64, 64),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(64, 100),
        )
#squeezenet = torchvision.models.squeezenet1_0()
#densenet = torchvision.models.densenet161()
#inception = torchvision.models.inception_v3(num_classes=10)

#net = resnet18.to(device)
#net = vgg16.to(device)
net = multi_layer_perceptron.to(device)
net.apply(models.weights_init)

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=lr)

## Training 

In [ ]:
train(net, trainloader, testloader, optimizer, criterion, n_epochs, classes=classes, verbose=True)
